In [12]:
import os
import torch
import random
import numpy as np
from loguru import logger
from torch.utils.data import DataLoader
from torch import nn

from pose.dataset import pose_dataset
from pose.utils import (
    collate_fn,
    geodesic_distance,
    relative_pose_error,
    relative_pose_error_np,
    recall_object
)
from pose.model import Mkpts_Reg_Model
from pose.animator import Animator


if os.name == 'nt':
    LM_dataset_path = 'd:/git_project/POPE/data/LM_dataset/'
    LM_dataset_json_path = 'd:/git_project/POPE/data/pairs/LINEMOD-test.json'
    LM_dataset_points_path = 'd:/git_project/POPE/data/LM_dataset-points/'

    onepose_path = 'e:/datasets/OnePose/test_data/'
    onepose_json_path = 'd:/git_project/POPE/data/pairs/Onepose-test.json'
    onepose_points_path = 'd:/git_project/POPE/data/onepose-points/'

    oneposeplusplus_path = 'e:/datasets/OnePose++/lowtexture_test_data/'
    oneposeplusplus_json_path = 'd:/git_project/POPE/data/pairs/OneposePlusPlus-test.json'
    oneposeplusplus_points_path = 'd:/git_project/POPE/data/oneposeplusplus-points/'
elif os.name == 'posix':
    LM_dataset_path = 'data/LM_dataset/'
    LM_dataset_json_path = 'data/pairs/LINEMOD-test.json'
    LM_dataset_points_path = 'data/LM_dataset-points/'

    onepose_path = 'data/onepose/'
    onepose_json_path = 'data/pairs/Onepose-test.json'
    onepose_points_path = 'data/onepose-points/'

    oneposeplusplus_path = 'data/oneposeplusplus/'
    oneposeplusplus_json_path = 'data/pairs/OneposePlusPlus-test.json'
    oneposeplusplus_points_path = 'data/oneposeplusplus-points/'

paths = [
    ('linemod', LM_dataset_path, LM_dataset_json_path, LM_dataset_points_path),
    # ('onepose', onepose_path, onepose_json_path, onepose_points_path),
    # ('oneposeplusplus', oneposeplusplus_path, oneposeplusplus_json_path, oneposeplusplus_points_path),
]

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

dataset = pose_dataset(paths)
mkpts_max_len, mkpts_sum_len = dataset.get_mkpts_info()

 62%|██████▏   | 8/13 [00:10<00:05,  1.19s/it]

d:/git_project/POPE/data/LM_dataset-points/0811-lm11-others\mkpts0\844.png-736.png.txt does not exist
d:/git_project/POPE/data/LM_dataset-points/0811-lm11-others\mkpts0\142.png-144.png.txt does not exist
d:/git_project/POPE/data/LM_dataset-points/0811-lm11-others\mkpts0\860.png-873.png.txt does not exist
d:/git_project/POPE/data/LM_dataset-points/0811-lm11-others\mkpts0\142.png-111.png.txt does not exist
d:/git_project/POPE/data/LM_dataset-points/0811-lm11-others\mkpts0\841.png-485.png.txt does not exist


 69%|██████▉   | 9/13 [00:11<00:04,  1.21s/it]

d:/git_project/POPE/data/LM_dataset-points/0812-lm12-others\mkpts0\582.png-581.png.txt does not exist


100%|██████████| 13/13 [00:15<00:00,  1.23s/it]


In [13]:
random.seed(20231223)
torch.manual_seed(20231223)
torch.cuda.manual_seed(20231223)

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

In [14]:
num_sample = 500
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True, drop_last=True, collate_fn=collate_fn(num_sample))
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=True, drop_last=True, collate_fn=collate_fn(num_sample))

In [15]:
net = torch.load('./weights/linemod-relative_r-gt_t-6d-500-2024-01-08-13-02-42-1.6629.pth').to(device)

net.eval()

Mkpts_Reg_Model(
  (embedding): Embedding()
  (transformerlayer): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=76, out_features=76, bias=True)
    )
    (linear1): Linear(in_features=76, out_features=2048, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (linear2): Linear(in_features=2048, out_features=76, bias=True)
    (norm1): LayerNorm((76,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((76,), eps=1e-05, elementwise_affine=True)
    (dropout1): Dropout(p=0.1, inplace=False)
    (dropout2): Dropout(p=0.1, inplace=False)
  )
  (transformer): TransformerEncoder(
    (layers): ModuleList(
      (0-3): 4 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=76, out_features=76, bias=True)
        )
        (linear1): Linear(in_features=76, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inp

In [16]:
id2name_dict = {
    1: "ape",
    2: "benchvise",
    4: "camera",
    5: "can",
    6: "cat",
    8: "driller",
    9: "duck",
    10: "eggbox",
    11: "glue",
    12: "holepuncher",
    13: "iron",
    14: "lamp",
    15: "phone",
}

In [17]:
# linemod
ape_data = []
benchvise_data = []
camera_data = []
can_data = []
cat_data = []
driller_data = []
duck_data = []
eggbox_data = []
glue_data = []
holepuncher_data = []
iron_data = []
lamp_data = []
phone_data = []
# onepose
aptamil_data = []
jzhg_data = []
minipuff_data = []
hlyormosiapie_data = []
brownhouse_data = []
oreo_data = []
mfmilkcake_data = []
diycookies_data = []
taipingcookies_data = []
tee_data = []
# onepose++
toyrobot_data = []
yellowduck_data = []
sheep_data = []
fakebanana_data = []
teabox_data = []
orange_data = []
greenteapot_data = []
lecreusetcup_data = []
insta_data = []

all_data = {
    # linemod
    'ape_data': ape_data,
    'benchvise_data': benchvise_data,
    'camera_data': camera_data,
    'can_data': can_data,
    'cat_data': cat_data,
    'driller_data': driller_data,
    'duck_data': duck_data,
    'eggbox_data': eggbox_data,
    'glue_data': glue_data,
    'holepuncher_data': holepuncher_data,
    'iron_data': iron_data,
    'lamp_data': lamp_data,
    'phone_data': phone_data,
    # onepose
    'aptamil_data': aptamil_data,
    'jzhg_data': jzhg_data,
    'minipuff_data': minipuff_data,
    'hlyormosiapie_data': hlyormosiapie_data,
    'brownhouse_data': brownhouse_data,
    'oreo_data': oreo_data,
    'mfmilkcake_data': mfmilkcake_data,
    'diycookies_data': diycookies_data,
    'taipingcookies_data': taipingcookies_data,
    'tee_data': tee_data,
    # onepose++
    'toyrobot_data': toyrobot_data,
    'yellowduck_data': yellowduck_data,
    'sheep_data': sheep_data,
    'fakebanana_data': fakebanana_data,
    'teabox_data': teabox_data,
    'orange_data': orange_data,
    'greenteapot_data': greenteapot_data,
    'lecreusetcup_data': lecreusetcup_data,
    'insta_data': insta_data,
}

In [18]:
linemod_type = ['ape_data', 'benchvise_data', 'camera_data', 'can_data', 'cat_data', 'driller_data', 'duck_data', 'eggbox_data', 'glue_data', 'holepuncher_data', 'iron_data', 'lamp_data', 'phone_data']
onepose_type = ['aptamil_data', 'jzhg_data', 'minipuff_data', 'hlyormosiapie_data', 'brownhouse_data', 'oreo_data', 'mfmilkcake_data', 'diycookies_data', 'taipingcookies_data', 'tee_data']
oneposeplusplus_type = ['toyrobot_data', 'yellowduck_data', 'sheep_data', 'fakebanana_data', 'teabox_data', 'orange_data', 'greenteapot_data', 'lecreusetcup_data', 'insta_data']

In [19]:
for i, batch in enumerate(test_dataloader):
    for data in batch:
        if 'lm' in data['name']:
            all_data[f"{id2name_dict[int(data['name'][2:])]}_data"].append(data)
        else:
            all_data[f"{data['name']}_data"].append(data)

empty_keys = []
for key in all_data.keys():
    if len(all_data[key]) == 0:
        empty_keys.append(key)

for key in empty_keys:
    all_data.pop(key)

for key in all_data.keys():
    print(key, len(all_data[key]))

print('len(all_data):', len(all_data))

ape_data 102
benchvise_data 71
camera_data 125
can_data 86
cat_data 100
driller_data 98
duck_data 95
eggbox_data 50
glue_data 106
holepuncher_data 101
iron_data 45
lamp_data 84
phone_data 89
len(all_data) 13


In [9]:
res_table = []

model_type = 'relative_r-gt_t'

for key in all_data.keys():
    if key in linemod_type:
        logger.info(f"LINEMOD: {key}")
    elif key in onepose_type:
        logger.info(f"ONEPOSE: {key}")
    elif key in oneposeplusplus_type:
        logger.info(f"ONEPOSE++: {key}")
    metrics = dict()
    metrics.update({'R_errs':[], 't_errs':[], 'inliers':[], "identifiers":[]})
    recall_image, all_image = 0, 0
    for item in all_data[key]:
        all_image += 1
        K0 = item['K0']
        K1 = item['K1']
        pose0 = item['pose0']
        pose1 = item['pose1']
        pre_bbox = item['pre_bbox']
        gt_bbox = item['gt_bbox']
        mkpts0 = item['mkpts0']
        mkpts1 = item['mkpts1']
        pre_K = item['pre_K']
        name = item['name']
        pair_name = item['pair_name']
        if 'lm' in name:
            name = id2name_dict[int(name[2:])]
        if name not in key:
            print(f'name: {name}, key: {key}')
            continue

        is_recalled = recall_object(pre_bbox, gt_bbox)

        recall_image = recall_image + int(is_recalled > 0.5)

        batch_mkpts0 = torch.from_numpy(mkpts0).unsqueeze(0).float().to(device)
        batch_mkpts1 = torch.from_numpy(mkpts1).unsqueeze(0).float().to(device)
        pre_t, pre_rot = net(batch_mkpts0, batch_mkpts1)
        pre_t = pre_t.squeeze(0).detach().cpu().numpy()
        pre_rot = pre_rot.squeeze(0).detach().cpu().numpy()

        if model_type == 'gt':
            t_err, R_err = relative_pose_error_np(pose1, pre_rot, pre_t, ignore_gt_t_thr=0.0)
        elif model_type == 'relative':
            relative_pose = np.matmul(pose1, np.linalg.inv(pose0))
            t_err, R_err = relative_pose_error_np(relative_pose, pre_rot, pre_t, ignore_gt_t_thr=0.0)
        elif model_type == 'relative_r-gt_t':
            relative_pose = np.matmul(pose1, np.linalg.inv(pose0))
            gt_pose = np.zeros_like(pose1)
            gt_pose[:3, :3] = relative_pose[:3, :3]
            gt_pose[:3, 3] = pose1[:3, 3]
            t_err, R_err = relative_pose_error_np(gt_pose, pre_rot, pre_t, ignore_gt_t_thr=0.0)

        metrics['R_errs'].append(R_err)
        metrics['t_errs'].append(t_err)
        metrics['identifiers'].append(pair_name)

    print(f"Acc: {recall_image}/{all_image}")
    import pprint
    from src.utils.metrics import (
        aggregate_metrics
    )
    from loguru import logger
    val_metrics_4tb = aggregate_metrics(metrics, 5e-4)
    val_metrics_4tb["AP50"] = recall_image / all_image
    logger.info('\n' + pprint.pformat(val_metrics_4tb))
    res_table.append([f"{name}"] + list(val_metrics_4tb.values()))

2024-01-08 18:25:53.742 | INFO     | __main__:<module>:7 - LINEMOD: ape_data


pre_t:
[ 0.03325049 -0.07125819  0.84045446]
gt_t:
[-0.13250454  0.03350499  0.79338747]
relative_t:
[0.79737847 0.19489724 0.38759566]
pre_t:
[0.003309   0.07818821 0.94732827]
gt_t:
[-0.22878009 -0.03225575  0.96876669]
relative_t:
[-0.02636355  0.10462934  0.3247855 ]
pre_t:
[ 0.0655687  -0.05722878  0.90606755]
gt_t:
[-0.01247146  0.14136802  0.86857212]
relative_t:
[-0.34623108 -0.55867123  0.68684115]
pre_t:
[ 0.03417    -0.0491489   0.80214715]
gt_t:
[-0.03973154  0.1757955   1.018682  ]
relative_t:
[ 0.29300905 -0.70840105  0.62918035]
pre_t:
[-0.05172608 -0.08968486  0.95460767]
gt_t:
[ 0.05826002 -0.07230116  0.80790097]
relative_t:
[-0.09401229 -0.88879636  0.82849514]
pre_t:
[ 0.03192649 -0.01357263  0.8399208 ]
gt_t:
[ 0.18148552 -0.03191694  0.94017565]
relative_t:
[-0.20008389 -0.30509212  0.28084449]
pre_t:
[ 0.01143481 -0.05350747  0.81917137]
gt_t:
[-0.10738782 -0.02690611  0.83542573]
relative_t:
[ 0.39638788 -0.62946606  1.56345192]
pre_t:
[ 0.02719515 -0.05594535  

2024-01-08 18:25:57.093 | INFO     | src.utils.metrics:aggregate_metrics:192 - Aggregating metrics over 102 unique items...
2024-01-08 18:25:57.095 | INFO     | __main__:<module>:71 - 
{'AP50': 0.6372549019607843,
 'R:ACC15': 0.00980392156862745,
 'R:ACC30': 0.1568627450980392,
 'R:auc@15': 0.007805654284870509,
 'R:auc@30': 0.04351889164697343,
 'R:medianErr': 66.19761793782445,
 't:ACC15': 0.9313725490196079,
 't:ACC30': 1.0,
 't:auc@15': 0.43515016702862364,
 't:auc@30': 0.7137732253403046,
 't:medianErr': 8.665881127078187}
2024-01-08 18:25:57.095 | INFO     | __main__:<module>:7 - LINEMOD: benchvise_data


pre_t:
[ 0.08460239 -0.02133844  0.8478926 ]
gt_t:
[ 0.03925488 -0.11446255  0.66365081]
relative_t:
[-0.57982678  0.03552358  0.10185835]
pre_t:
[-0.01234867 -0.02034062  0.9125377 ]
gt_t:
[0.14475785 0.0087794  1.02957964]
relative_t:
[-0.11079038 -0.28663833  0.51092958]
pre_t:
[ 0.02023221 -0.06103107  0.8650328 ]
gt_t:
[0.09863744 0.09088158 1.02459717]
relative_t:
[-0.43436263 -0.43755508  1.30257326]
pre_t:
[ 0.04092666 -0.06070082  0.8615607 ]
gt_t:
[-0.06534532  0.02682884  0.92463911]
relative_t:
[-0.30646527 -0.57397285  0.9588506 ]
pre_t:
[ 0.01832808 -0.04512085  0.8999933 ]
gt_t:
[-0.03483285 -0.04023927  0.6915099 ]
relative_t:
[-0.67884716  0.06068082  0.32926931]
pre_t:
[4.2070448e-04 2.6755624e-02 9.0189654e-01]
gt_t:
[ 0.02976879 -0.01674361  0.98377091]
relative_t:
[ 0.07870379 -0.56431284  1.39005994]
pre_t:
[-0.04633661 -0.0943611   0.8626997 ]
gt_t:
[0.03744414 0.04485041 0.78113192]
relative_t:
[-0.03558685  0.33797916  0.19675689]
pre_t:
[ 0.05029139 -0.0374708

2024-01-08 18:25:57.773 | INFO     | src.utils.metrics:aggregate_metrics:192 - Aggregating metrics over 71 unique items...
2024-01-08 18:25:57.775 | INFO     | __main__:<module>:71 - 
{'AP50': 0.3380281690140845,
 'R:ACC15': 0.014084507042253521,
 'R:ACC30': 0.11267605633802817,
 'R:auc@15': 0.007731367806768512,
 'R:auc@30': 0.04191436602460065,
 'R:medianErr': 90.45960979109566,
 't:ACC15': 1.0,
 't:ACC30': 1.0,
 't:auc@15': 0.5747023887278764,
 't:auc@30': 0.7873511943639381,
 't:medianErr': 6.1466996162332865}
2024-01-08 18:25:57.776 | INFO     | __main__:<module>:7 - LINEMOD: camera_data


...

pre_t:
[ 0.03289165 -0.0379747   0.83173573]
gt_t:
[ 0.00220194 -0.17555639  0.90378082]
relative_t:
[-0.75828465 -0.55238634  0.58230131]
Acc: 20/45
pre_t:
[-0.04158088 -0.05433287  0.85983944]
gt_t:
[ 0.06293269 -0.11990828  0.86672729]
relative_t:
[ 1.06440869 -0.16107166  0.60828228]
pre_t:
[ 0.03870953 -0.03603489  0.7741514 ]
gt_t:
[ 0.06293269 -0.11990828  0.86672729]
relative_t:
[-0.11704568 -0.90327229  0.61343271]
pre_t:
[ 0.03345352 -0.06555639  0.8890752 ]
gt_t:
[-0.00646472 -0.03550887  0.75419533]
relative_t:
[-0.37677187 -0.57566752  0.36337593]
pre_t:
[ 0.04973909 -0.04570401  0.8332126 ]
gt_t:
[ 0.05628709 -0.00485256  0.87217116]
relative_t:
[0.71625324 0.12539254 0.20998601]
pre_t:
[ 0.0501378  -0.05628039  0.81206495]
gt_t:
[ 0.02407844 -0.12187865  0.98431665]
relative_t:
[-0.7935992  -0.67676922  1.3407101 ]
pre_t:
[ 0.02388703 -0.02365085  0.9360677 ]
gt_t:
[-0.05944727 -0.09424346  0.6877327 ]
relative_t:
[-0.84062992 -0.46884636  0.84156825]
pre_t:
[ 0.04405846

2024-01-08 18:26:06.209 | INFO     | src.utils.metrics:aggregate_metrics:192 - Aggregating metrics over 83 unique items...
2024-01-08 18:26:06.211 | INFO     | __main__:<module>:71 - 
{'AP50': 0.44047619047619047,
 'R:ACC15': 0.011904761904761904,
 'R:ACC30': 0.16666666666666666,
 'R:auc@15': 0.008690858457737616,
 'R:auc@30': 0.052937082191625504,
 'R:medianErr': 76.65179379587128,
 't:ACC15': 0.9880952380952381,
 't:ACC30': 1.0,
 't:auc@15': 0.5699096240533815,
 't:auc@30': 0.7850559530878033,
 't:medianErr': 5.995505908623905}
2024-01-08 18:26:06.212 | INFO     | __main__:<module>:7 - LINEMOD: phone_data


pre_t:
[-0.00284237 -0.03827496  0.87389433]
gt_t:
[-0.03543714 -0.06364609  0.73405546]
relative_t:
[ 0.68720102 -0.32636437  0.01450443]
pre_t:
[ 0.05070857 -0.07924861  0.8300471 ]
gt_t:
[-0.04868753 -0.16056868  0.85253555]
relative_t:
[-0.01677914 -0.51912512  0.33078806]
pre_t:
[ 0.0440461  -0.02853968  0.94791716]
gt_t:
[-0.14722988 -0.05794331  1.03325641]
relative_t:
[0.52310959 0.08620576 0.27397629]
pre_t:
[-0.02097519 -0.06944832  0.8528161 ]
gt_t:
[0.09840423 0.03927929 0.9829582 ]
relative_t:
[ 1.02222323 -0.28278595  0.69198357]
pre_t:
[ 0.06213376 -0.01742427  0.9052592 ]
gt_t:
[ 0.11523277 -0.05167022  0.6415264 ]
relative_t:
[-0.04647344 -0.57181059 -0.20868021]
pre_t:
[ 0.02143456 -0.0231365   0.804458  ]
gt_t:
[-0.02766579  0.05353888  1.03075457]
relative_t:
[-0.2697094  -0.53968676  1.06359864]
pre_t:
[ 0.03427361 -0.04516823  0.78870684]
gt_t:
[ 0.03011    -0.10762621  0.67198336]
relative_t:
[ 0.0603273  -0.74122353  0.99892786]
pre_t:
[ 0.04917078 -0.04324688  

2024-01-08 18:26:07.042 | INFO     | src.utils.metrics:aggregate_metrics:192 - Aggregating metrics over 89 unique items...
2024-01-08 18:26:07.044 | INFO     | __main__:<module>:71 - 
{'AP50': 0.33707865168539325,
 'R:ACC15': 0.02247191011235955,
 'R:ACC30': 0.06741573033707865,
 'R:auc@15': 0.009722257995613765,
 'R:auc@30': 0.0270744558230742,
 'R:medianErr': 63.244846728456174,
 't:ACC15': 1.0,
 't:ACC30': 1.0,
 't:auc@15': 0.5820350361176766,
 't:auc@30': 0.7910175180588382,
 't:medianErr': 6.001364306984559}


pre_t:
[ 0.07578838 -0.04912034  0.8705179 ]
gt_t:
[ 0.02073299 -0.01880141  1.00287044]
relative_t:
[-0.63895971 -0.0638043   0.74656474]
pre_t:
[ 0.07016094 -0.01175529  0.8484143 ]
gt_t:
[ 0.30521265 -0.06345942  0.99623257]
relative_t:
[ 0.05309722 -0.62940464  0.39182842]
pre_t:
[0.00817125 0.02008631 0.905445  ]
gt_t:
[-0.0064622  -0.10310427  1.03794229]
relative_t:
[-0.45155877 -0.04507274  0.13954002]
pre_t:
[-0.02791081 -0.07842454  0.82456076]
gt_t:
[-0.0073779  -0.08888417  0.99985284]
relative_t:
[0.93919507 0.11583653 0.53212988]
pre_t:
[ 0.04852004 -0.06476907  0.84239864]
gt_t:
[0.02990725 0.04220477 0.85955149]
relative_t:
[ 0.4216581  -0.40966789  0.35015944]
pre_t:
[ 0.07879925 -0.05749088  0.76832175]
gt_t:
[-0.12342578  0.02391978  1.03365493]
relative_t:
[-0.04707529 -0.87845502  1.53975161]
pre_t:
[-0.00546962 -0.03104579  0.8449845 ]
gt_t:
[-0.07509425 -0.01384886  1.003124  ]
relative_t:
[-0.38694965 -0.57823912  0.72458268]
pre_t:
[ 0.02289686 -0.09667017  0.8

In [10]:
from tabulate import tabulate
headers = ["Category"] + list(val_metrics_4tb.keys())
all_data = np.array(res_table)[:, 1:].astype(np.float32)
res_table.append(["Avg"] + all_data.mean(0).tolist())
print(tabulate(res_table, \
    headers=headers, tablefmt='fancy_grid'))

╒═════════════╤════════════╤════════════╤════════════╤═══════════╤═══════════════╤════════════╤════════════╤═══════════╤═══════════╤═══════════════╤══════════╕
│ Category    │   R:auc@15 │   R:auc@30 │    R:ACC15 │   R:ACC30 │   R:medianErr │   t:auc@15 │   t:auc@30 │   t:ACC15 │   t:ACC30 │   t:medianErr │     AP50 │
╞═════════════╪════════════╪════════════╪════════════╪═══════════╪═══════════════╪════════════╪════════════╪═══════════╪═══════════╪═══════════════╪══════════╡
│ ape         │ 0.00780565 │  0.0435189 │ 0.00980392 │ 0.156863  │       66.1976 │   0.43515  │   0.713773 │  0.931373 │         1 │       8.66588 │ 0.637255 │
├─────────────┼────────────┼────────────┼────────────┼───────────┼───────────────┼────────────┼────────────┼───────────┼───────────┼───────────────┼──────────┤
│ benchvise   │ 0.00773137 │  0.0419144 │ 0.0140845  │ 0.112676  │       90.4596 │   0.574702 │   0.787351 │  1        │         1 │       6.1467  │ 0.338028 │
├─────────────┼────────────┼────────────